# PuLP w/ custom utils practical demo

LP problem is adapted from a homework solution made with Julia and JuMP. Custom utils allow us to leverage np.ndarrays with casing, indexing, sums etc.

In [1]:
from dataclasses import dataclass
import numpy as np
from pulp import LpMinimize
from util import LpProblemIterable, lpvar

In [2]:
@dataclass
class ProblemInstance:
    """Container for problem data."""

    ni: int     # Number of suppliers      
    nj: int     # Number of demand points
    nt: int     # Number of periods
    c: np.ndarray      # Unit capacity costs per supplier
    h: np.ndarray      # Unit storage cost per supplier
    m: np.ndarray      # Production cost per supplier
    d: np.ndarray      # Client demands in all periods
    q: np.ndarray      # Unit costs of unfulfilled demand
    f: np.ndarray      # Unit costs to fulfil demands


In [3]:
# initialise test instance

ni = 5
nj = 5
nt = 5
c = np.ones((ni, 1)) * 5
h = np.ones((ni, 1)) * 0.1
m = np.ones((ni, 1))

d = np.zeros((nj, nt))
for j in range(nj):
    for t in range(nt):
        d[j, t] = (j + 1) + 0.05 * t

q = np.ones((nj, 1)) * 50

f = np.zeros((ni, nj, 1))
for i in range(ni):
    for j in range(nj):
        f[i, j] = abs(i - j)

test_instance = ProblemInstance(ni, nj, nt, c, h, m, d, q, f)

In [4]:
def solve_instance(ins: ProblemInstance):
    # unpack problem definition from instance
    ni = ins.ni
    nj = ins.nj
    nt = ins.nt
    c = ins.c
    h = ins.h
    m = ins.m
    d = ins.d
    q = ins.q
    f = ins.f

    # model definition
    model = LpProblemIterable(name='supply-chain', sense=LpMinimize)

    # variable definitions
    lb = dict(lowBound=0)
    x = lpvar('x', ni, 1, **lb)
    p = lpvar('p', ni, nt, **lb)
    k = lpvar('k', ni, nt + 1, **lb)
    e = lpvar('e', ni, nj, nt, **lb)
    u = lpvar('u', nj, nt, **lb)

    # objective definition
    obj = (
        (c * x).sum() +
        (h * k[:, 1:] + m * p).sum() +
        (f * e).sum() +
        (q * u).sum()
    )
    model += obj

    # constraints
    model += p <= x
    model += p + k[:, :-1] == e.sum(axis=1) + k[:, 1:]
    model += e.sum(axis=0) == d - u
    model += k[:, 0] == 0

    # solve and return solved instance
    model.solve()
    return model

In [5]:
solved_model = solve_instance(test_instance)
solved_model.sol_status

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/leevi/.pyenv/versions/3.10.4/envs/py-linopt/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nc/dhtws2m1317bwzbsxq9n3_gc0000gn/T/a9874e3b2f3846d5aa9ce74bb74f15fa-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nc/dhtws2m1317bwzbsxq9n3_gc0000gn/T/a9874e3b2f3846d5aa9ce74bb74f15fa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 85 COLUMNS
At line 671 RHS
At line 752 BOUNDS
At line 753 ENDATA
Problem MODEL has 80 rows, 210 columns and 405 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 70 (-10) rows, 200 (-10) columns and 385 (-20) elements
Perturbing problem by 0.001% of 50 - largest nonzero change 0.00019720225 ( 0.075142408%) - largest zero change 7.2486756e-05
0  Obj 0 Primal inf 77.499975 (25)
33  Obj 74.045729 Primal inf 75.449978 (22)
65  Obj 1

1

In [6]:
solved_model.objective.value()

155.24999999999986

In [ ]:
for var in solved_model.variables():
    if var.name.startswith('x'):
        print(f'{var.name}: {var.value()}')

x_0_0: 1.1
x_1_0: 2.1
x_2_0: 3.1
x_3_0: 4.1
x_4_0: 5.1
